***
**Importing necessary libraries**
***

In [ ]:
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf                
from tqdm import tqdm

***
**Merging Google Drive**
***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


***
**Dataset Path**
***

In [ ]:
dataset='/content/drive/MyDrive/BRAC /Past Courses/Thesis/Thesis Final/Dataset/CNN'

***
**Installing and importing mne for reading EEG data**
***

In [ ]:
!pip install mne

In [ ]:
import mne

***
**Creating Classes**
***


In [ ]:
class_names = ['Healthy', 'MDD']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

***
**Data Preprocessing**
***

In [ ]:
images = []
labels = []
output=[]
IMAGE_SIZE = (150, 150)
print("Loading {}".format(dataset))

# Iterate through each folder corresponding to a category
for folder in os.listdir(dataset):
    label = class_names_label[folder]
    
    # Iterate through each image in our folder
    for file in tqdm(os.listdir(os.path.join(dataset, folder))):
        
        # Get the path name of the image
        img_path = os.path.join(os.path.join(dataset, folder), file)

        # Open and resize the img
        image = cv2.imread(img_path)
        image = image[6:208,92:420]
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, IMAGE_SIZE) 
        
        # Append the image and its corresponding label to the output
        images.append(image)
        labels.append(label)

images = np.array(images, dtype = 'float32')
labels = np.array(labels, dtype = 'int32')

output.append((images, labels))

Loading /content/drive/MyDrive/BRAC /Past Courses/Thesis/Thesis Final/Dataset/CNN


100%|██████████| 710/710 [00:05<00:00, 140.80it/s]


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.15, random_state=42)

***
**Creating the 2D CNN model**
***

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 32)        9

***
**Accuracy Level**
***

In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

***
**Data Training**
***

In [ ]:
history = model.fit(X_test, y_test, batch_size=30, epochs=50)

Epoch 1/50
7/7 [==============================] - 6s 810ms/step - loss: 0.1466 - accuracy: 0.9378
Epoch 2/50
7/7 [==============================] - 6s 805ms/step - loss: 0.1161 - accuracy: 0.9569
Epoch 3/50
7/7 [==============================] - 6s 818ms/step - loss: 0.1624 - accuracy: 0.9378
Epoch 4/50
7/7 [==============================] - 6s 808ms/step - loss: 0.1431 - accuracy: 0.9474
Epoch 5/50
7/7 [==============================] - 6s 805ms/step - loss: 0.1537 - accuracy: 0.9474
Epoch 6/50
7/7 [==============================] - 6s 806ms/step - loss: 0.1094 - accuracy: 0.9474
Epoch 7/50
7/7 [==============================] - 6s 807ms/step - loss: 0.1089 - accuracy: 0.9617
Epoch 8/50
7/7 [==============================] - 6s 817ms/step - loss: 0.2301 - accuracy: 0.9043
Epoch 9/50
7/7 [==============================] - 6s 809ms/step - loss: 0.3278 - accuracy: 0.8565
Epoch 10/50
7/7 [==============================] - 6s 806ms/step - loss: 0.4346 - accuracy: 0.8469
Epoch 11/50
7/7 [==